In [ ]:
from sympy import Point, Line

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
hailstones = [
    [
        (
            int(v.split(",")[0].strip()),
            int(v.split(",")[1].strip()),
            int(v.split(",")[2].strip()),
        )
        for v in l.split("@")
    ]
    for l in lines
]
hailstones[:5]

In [ ]:
def mv_pos_2d(pos, dir):
    return (pos[0] + dir[0], pos[1] + dir[1])

In [ ]:
test_min = 200000000000000
test_max = 400000000000000

intersect_counts = 0
for i in range(0, len(hailstones) - 1):
    h1 = hailstones[i]
    for j in range(i + 1, len(hailstones)):
        h2 = hailstones[j]

        h1_p1 = Point(h1[0][:2])
        h1_p2 = Point(mv_pos_2d(h1[0][:2], h1[1][:2]))
        h1_line = Line(h1_p1, h1_p2)

        h2_p1 = Point(h2[0][:2])
        h2_p2 = Point(mv_pos_2d(h2[0][:2], h2[1][:2]))
        h2_line = Line(h2_p1, h2_p2)

        intersection = h1_line.intersection(h2_line)

        if not intersection:
            continue

        if (
            intersection[0][0] < test_min
            or intersection[0][0] > test_max
            or intersection[0][1] < test_min
            or intersection[0][1] > test_max
        ):
            continue

        # print(h1_line, h2_line)
        # print(intersection)
        # print(Line(h1_p1, intersection[0]).angle_between(h1_line), Line(h2_p1, intersection[0]).angle_between(h2_line))
        if (
            Line(h1_p1, intersection[0]).angle_between(h1_line) == 0
            and Line(h2_p1, intersection[0]).angle_between(h2_line) == 0
        ):
            intersect_counts += 1
intersect_counts

part 2

Idea is to put it into a equation system like this (only for first 2 example hailstones):
The rock has the same start position and velocity for all hailstones and must share the same position for a given timestamp (l1, l2, ...) that can be different between hailstones.

Rock is r_v1 + l*r_v2

hailstone1    19, 13, 30 @ -2, 1, -2    h1_v1 = (19,13,30)  h1_v2 = (-2,1,-2)
hailstone2    18, 19, 22 @ -1, -1, -2   h2_v1 = (18,19,22)  h2_v2 = (-1,-1,-2)

h1_v1 + l1 * h1_v2 = r_v1 + l1 * r_v2
h2_v1 + l2 * h2_v2 = r_v1 + l2 * r_v2

19 + -2*l1 = r1_x + l1 * r2_x
13 +  1*l1 = r1_y + l1 * r2_y
30 + -2*l1 = r1_z + l1 * r2_z
18 + -1*l2 = r1_x + l2 * r2_x
19 + -1*l2 = r1_y + l2 * r2_y
22 + -2*l2 = r1_z + l2 * r2_z

And then solve it using a linear solver like z3

In [ ]:
from z3 import *

r_vars = Ints("r11 r12 r13 r21 r22 r23")
s = Solver()

for i in range(0, len(hailstones)):
    l_var = Ints(f"l{i}")[0]
    hs = hailstones[i]

    s.add(hs[0][0] + (hs[1][0] * l_var) == r_vars[0] + l_var * r_vars[3])
    s.add(hs[0][1] + (hs[1][1] * l_var) == r_vars[1] + l_var * r_vars[4])
    s.add(hs[0][2] + (hs[1][2] * l_var) == r_vars[2] + l_var * r_vars[5])

# print(s.sexpr())

In [ ]:
s.check()
model = s.model()
# print(model)
int(str(model[r_vars[0]])) + int(str(model[r_vars[1]])) + int(str(model[r_vars[2]]))